In [1]:
#Imports
import pandas as pd # data processing for CSV and SQLLite
import sqlite3 
import timeit 
import requests 
import io
import opendatasets as od ## kaggle API
start_time = timeit.default_timer()

## SQLLite DB California Vehicle Collison Data (2001-2021)
## Business Use Case: State Government, Car Insurance, Police Force, etc.

try:
    od.download("https://www.kaggle.com/alexgude/california-traffic-collision-data-from-switrs") ##download SQLLite DB
except Exception as err:
    print("API Error")
    print(err)
    
path = "your path here"   ##path to downloaded DB
database = path + 'switrs.sqlite'


Skipping, found downloaded files in "./california-traffic-collision-data-from-switrs" (use force=True to force download)


In [2]:
conn = sqlite3.connect(database)

tables = pd.read_sql('''SELECT *
                        FROM sqlite_master
                        WHERE type='table';''', conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,case_ids,case_ids,2,"CREATE TABLE case_ids(case_id TEXT,db_year)"
1,table,collisions,collisions,50067,"CREATE TABLE collisions(\n case_id TEXT,\n j..."
2,table,victims,victims,1060817,"CREATE TABLE victims(\n id INT,\n case_id TE..."
3,table,parties,parties,1339384,"CREATE TABLE parties(\n id INT,\n case_id TE..."


In [3]:
collisions_table = pd.read_sql('''SELECT *
                        FROM collisions
                        ORDER BY collision_date DESC
                        LIMIT 10;''', conn)
collisions_table

,case_id,jurisdiction,officer_id,reporting_district,chp_shift,population,county_city_location,county_location,special_condition,beat_type,...,bicyclist_injured_count,motorcyclist_killed_count,motorcyclist_injured_count,primary_ramp,secondary_ramp,latitude,longitude,collision_date,collision_time,process_date
0,91488723,9580,016413,None,0600 thru 1359,>250000,1942,los angeles,0,chp state highway,...,0,0,0,None,None,34.16573,-118.48442,2021-06-03,09:55:00,2021-06-03
1,91488958,9525,018295,None,2200 thru 0559,10000 to 25000,1921,los angeles,0,chp state highway,...,0,0,0,None,None,34.13552,-117.97649,2021-06-03,05:15:00,2021-06-03
2,81488159,704,0583,5,not chp,100000 to 250000,0704,contra costa,0,not chp,...,0,0,0,None,None,37.96389,-121.99281,2021-06-02,21:50:00,2021-06-03
3,81488202,1942,32973,1361,not chp,>250000,1942,los angeles,0,not chp,...,0,0,1,None,None,NaN,NaN,2021-06-02,07:30:00,2021-06-03
4,81488208,1942,39508,0411,not chp,>250000,1942,los angeles,0,not chp,...,0,0,0,None,None,NaN,NaN,2021-06-02,02:10:00,2021-06-03
5,81488214,1942,37143,0415,not chp,>250000,1942,los angeles,0,not chp,...,0,0,0,None,None,NaN,NaN,2021-06-02,15:48:00,2021-06-03
6,81488218,1942,36814,1341,not chp,>250000,1942,los angeles,0,not chp,...,0,0,0,None,None,NaN,NaN,2021-06-02,21:40:00,2021-06-03
7,81488236,1942,36238,1353,not chp,>250000,1942,los angeles,0,not chp,...,0,0,0,None,None,NaN,NaN,2021-06-02,07:30:00,2021-06-03
8,81488262,1942,42932,0212,not chp,>250000,1942,los angeles,0,not chp,...,0,0,0,None,None,NaN,NaN,2021-06-02,09:00:00,2021-06-03
9,81488270,1942,42932,None,not chp,>250000,1942,los angeles,0,not chp,...,0,0,0,None,None,NaN,NaN,2021-06-02,12:00:00,2021-06-03


In [4]:
case_ids = pd.read_sql('''SELECT *
                        FROM case_ids
                        LIMIT 10;''', conn)
case_ids

,case_id,db_year
0,0081715,2021
1,0726202,2021
2,3858022,2021
3,3899441,2021
4,3899442,2021
5,3899445,2021
6,3899446,2021
7,3899449,2021
8,3899450,2021
9,3899453,2021


In [5]:
victims = pd.read_sql('''SELECT *
                        FROM victims
                        LIMIT 10;''', conn)
victims

,id,case_id,party_number,victim_role,victim_sex,victim_age,victim_degree_of_injury,victim_seating_position,victim_safety_equipment_1,victim_safety_equipment_2,victim_ejected
0,1,3858022,1,passenger,female,24,no injury,passenger seat 4,air bag not deployed,None,not ejected
1,2,3858022,1,passenger,male,31,no injury,passenger seat 6,air bag not deployed,None,not ejected
2,3,3899450,1,driver,female,72,complaint of pain,driver,air bag not deployed,lap/shoulder harness used,not ejected
3,4,3899453,2,driver,male,66,complaint of pain,driver,air bag deployed,lap/shoulder harness used,not ejected
4,5,3899453,3,passenger,male,65,no injury,passenger seat 3,air bag not deployed,lap/shoulder harness used,not ejected
5,6,3899454,1,pedestrian,female,29,complaint of pain,position unknown,None,not required,unknown
6,7,3899454,2,pedestrian,female,7,complaint of pain,position unknown,None,not required,unknown
7,8,3899457,1,passenger,female,60,no injury,passenger seat 3,air bag not deployed,lap/shoulder harness used,not ejected
8,9,3899457,2,passenger,female,1,no injury,passenger seat 6,air bag not deployed,child restraint in vehicle used,not ejected
9,10,3899458,1,passenger,male,19,no injury,passenger seat 3,air bag not deployed,lap/shoulder harness used,not ejected


In [6]:
parties = pd.read_sql('''SELECT *
                        FROM parties
                        LIMIT 10;''', conn)
parties

,id,case_id,party_number,party_type,at_fault,party_sex,party_age,party_sobriety,party_drug_physical,direction_of_travel,...,other_associate_factor_2,party_number_killed,party_number_injured,movement_preceding_collision,vehicle_year,vehicle_make,statewide_vehicle_type,chp_vehicle_type_towing,chp_vehicle_type_towed,party_race
0,1,0081715,1,driver,1,female,35.0,not applicable,not applicable,north,...,None,0,0,proceeding straight,2007,ford,None,None,None,other
1,2,0081715,2,driver,0,female,43.0,not applicable,not applicable,north,...,None,0,0,proceeding straight,2019,None,None,None,None,hispanic
2,3,0726202,1,driver,1,None,NaN,impairment unknown,G,north,...,None,0,0,proceeding straight,2005,None,passenger car,"passenger car, station",None,None
3,4,3858022,1,driver,1,male,47.0,impairment unknown,G,east,...,None,0,0,None,2007,dodge,emergency vehicle,police car,00,black
4,5,3858022,2,driver,0,male,58.0,had not been drinking,None,east,...,None,0,0,None,2006,nissan,None,None,00,hispanic
5,6,3899441,1,driver,1,male,16.0,impairment unknown,G,south,...,None,0,0,proceeding straight,1994,toyota,passenger car,"passenger car, station",00,asian
6,7,3899442,1,driver,1,male,40.0,had not been drinking,None,south,...,None,0,0,proceeding straight,2001,toyota,passenger car,"passenger car, station",00,hispanic
7,8,3899442,2,driver,0,male,46.0,had not been drinking,None,south,...,None,0,0,proceeding straight,2003,toyota,passenger car,"passenger car, station",00,asian
8,9,3899445,1,driver,1,male,24.0,had not been drinking,None,north,...,None,0,0,other,1999,toyota,passenger car,"passenger car, station",00,hispanic
9,10,3899445,2,driver,0,female,32.0,had not been drinking,None,north,...,None,0,0,proceeding straight,2007,nissan,passenger car,"passenger car, station",00,asian


In [7]:
## HOW MANY COLLISIONS ARE IN THE DATA SET? == 9,424,334
num_of_collisions = pd.read_sql('''SELECT COUNT()
                        FROM collisions''', conn)
num_of_collisions

,COUNT()
0,9424334


In [8]:
## WHERE HAVE MOST OF THE COLLISIONS HAPPENED IN CALIFORNIA
## SHOW ON TABLEAU MAP ##

area_of_collision = pd.read_sql('''SELECT county_location,COUNT(*) as count,latitude,longitude
    FROM collisions 
    GROUP BY county_location 
    HAVING count(*)>1
    ORDER BY count DESC''', conn)


area_of_collision

,county_location,count,latitude,longitude
0,los angeles,2851925,NaN,NaN
1,orange,728565,NaN,NaN
2,san bernardino,569376,NaN,NaN
3,san diego,535596,NaN,NaN
4,riverside,493758,NaN,NaN
5,alameda,466969,NaN,NaN
6,sacramento,403575,NaN,NaN
7,santa clara,342603,NaN,NaN
8,contra costa,217025,NaN,NaN
9,san joaquin,209185,NaN,NaN


In [9]:
##COUNT OF COLLISIONS PER YEAR
collisions_per_year = pd.read_sql('''
SELECT count(*) as count_of_collisions, 
        strftime('%Y',collision_date) as year  
FROM collisions
GROUP BY year
ORDER BY year
''', conn)
collisions_per_year

,count_of_collisions,year
0,522562,2001
1,544741,2002
2,538954,2003
3,538295,2004
4,532725,2005
5,498846,2006
6,501908,2007
7,452595,2008
8,426228,2009
9,416490,2010


In [10]:
##NUMBER OF COLLISIONS PER MONTH

collisions_per_month = pd.read_sql('''
SELECT count(*) as count_of_collisions, 
        strftime('%m',collision_date) as month  
FROM collisions
GROUP BY month
ORDER BY month 
''', conn)
collisions_per_month


,count_of_collisions,month
0,783501,01
1,758128,02
2,822566,03
3,770941,04
4,793315,05
5,750901,06
6,757007,07
7,774338,08
8,779719,09
9,838819,10


In [11]:
# PERCENTAGE OF MALE VS FEMALES DRIVERS INVOLVED IN COLLISIONS INCLUDING UNKNOWN

percent_collisions = pd.read_sql('''
SELECT ( SELECT
    COUNT(case_id) 
    / (
        SELECT CAST(COUNT(case_id) AS FLOAT) 
        FROM parties 
        )
    AS percentage
FROM parties
WHERE party_sex IS NULL ) AS unknown_percentage,
( SELECT 
    COUNT(case_id) 
    / (
        SELECT CAST(COUNT(case_id) AS FLOAT) 
        FROM parties 
        )
    AS percentage
FROM parties
WHERE party_sex='female') AS female_percentage,
( SELECT 
    COUNT(case_id) 
    / (
        SELECT CAST(COUNT(case_id) AS FLOAT) 
        FROM parties 
        )
    AS percentage
FROM parties
WHERE party_sex='male') AS male_percentage
''', conn)
percent_collisions


,unknown_percentage,female_percentage,male_percentage
0,0.133859,0.335217,0.530708


In [12]:
##Compare Driver Stats with actual sex demographics data from California
##California Demographic Statistics
#Getting Female percent. Tableau will do the calculation for male percentage
##This demographics data is up to date as of July 1st, 2021 https://www.census.gov/quickfacts/CA

csv_response = "https://www.census.gov/quickfacts/fact/csv/CA/PST045221"
try:
    resp = requests.get(csv_response)
except Exception as err:
    print("API Error")
    print(err)
df=pd.read_csv(io.StringIO(resp.text), sep=',')
demographic_stats = df[['Fact', 'California']]
california_female_percentage = demographic_stats.loc[demographic_stats['Fact'] == "Female persons, percent"]
california_female_percentage

display(california_female_percentage)


,Fact,California
8,"Female persons, percent",50.3%


In [13]:
# HOW MANY COLLISIONS HAPPENED IN 2020? Did covid effect collisions? 
# USE DATA TO CHART ON TABLEAU

covid_collisions = pd.read_sql('''
SELECT case_id,collision_time,collision_date
FROM collisions
WHERE date(collision_date) BETWEEN date('2020-01-01') and date('2020-12-31')
ORDER BY collision_date ASC
''', conn)

covid_collisions

,case_id,collision_time,collision_date
0,81158280,00:40:00,2020-01-01
1,81158292,02:51:00,2020-01-01
2,81158296,20:04:00,2020-01-01
3,81158297,21:38:00,2020-01-01
4,81158306,22:49:00,2020-01-01
...,...,...,...
366412,9263838,05:32:00,2020-12-31
366413,9263840,09:40:00,2020-12-31
366414,9264216,14:30:00,2020-12-31
366415,9265217,18:27:00,2020-12-31


In [14]:
# DRIVERS UNDER THE INFLUENCE BASED ON DAY PER WEEK
underinfluence_dow = pd.read_sql('''
WITH CTE AS (
    SELECT parties.party_sobriety, 
            strftime('%w',collisions.collision_date) as day_of_week  
    FROM parties
    JOIN collisions on parties.case_id = collisions.case_id
)
SELECT count(*) as count_of_dui,day_of_week 
FROM CTE
WHERE party_sobriety = 'had been drinking, under influence'
GROUP BY day_of_week
ORDER BY day_of_week 
''', conn)
underinfluence_dow

#0 Sunday,1 Monday

,count_of_dui,day_of_week
0,159910,0
1,71226,1
2,61229,2
3,66673,3
4,75380,4
5,107809,5
6,161718,6


In [15]:
# Average age of drivers who were at fault that involves another motor-vehicle
average_age_of_drivers = pd.read_sql('''
SELECT parties.party_age, 
        parties.party_sex 
from parties 
JOIN collisions ON parties.case_id = collisions.case_id
WHERE parties.at_fault = 1
    AND party_type = 'driver'
    AND collisions.motor_vehicle_involved_with  IN ('motor vehicle on other roadway', 'other motor vehicle')
    AND (COALESCE(party_age, party_sex) IS NOT NULL 
        OR COALESCE(party_age, party_sex) != '') 
''', conn)
average_age_of_drivers


,party_age,party_sex
0,35.0,female
1,47.0,male
2,40.0,male
3,24.0,male
4,36.0,male
...,...,...
5244616,66.0,male
5244617,21.0,female
5244618,14.0,female
5244619,17.0,female


In [16]:
#CREATE EXCEL FILES

# HOW MANY COLLISIONS ARE IN THE DATA SET? 

num_of_collisions.to_excel("output/num_of_collisions.xlsx",
            sheet_name='num_of_collisions')  
print("Created num_of_collisions Excel")

# WHERE HAVE MOST OF THE COLLISIONS HAPPENED IN CALIFORNIA
# SHOW ON TABLEAU MAP ##

area_of_collision.to_excel("output/area_of_collision.xlsx",
            sheet_name='area_of_collision')  
print("Created area_of_collision Excel")

# HOW MANY COLLISIONS HAPPENED IN 2020? Did covid effect collisions? 
# USE DATA TO CHART ON TABLEAU 

covid_collisions.to_csv("output/covid_collisions.csv")  
print("Created covid_collisions CSV")

# PERCENTAGE OF MALE VS FEMALES DRIVERS INVOLVED IN COLLISIONS INCLUDING UNKNOWN 

percent_collisions.to_excel("output/percent_collisions.xlsx",
            sheet_name='percent_collisions')  
print("Created percent_collisions Excel")

##Compare Driver Stats with actual MALE/FEMALE demographics data from California

california_female_percentage.to_excel("output/california_female_percentage.xlsx",
            sheet_name='california_female_percentage')  
print("Created california_female_percentage Excel")

##COUNT OF COLLISIONS PER YEAR 

collisions_per_year.to_excel("output/collisions_per_year.xlsx",
            sheet_name='collisions_per_year')  
print("Created collisions_per_year Excel")

##NUMBER OF COLLISIONS PER MONTH 

collisions_per_month.to_excel("output/collisions_per_month.xlsx",
            sheet_name='collisions_per_month')  
print("Created collisions_per_month Excel")

# DRIVERS UNDER THE INFLUENCE BASED ON DAY PER WEEK 

underinfluence_dow.to_excel("output/underinfluence_dow.xlsx",
            sheet_name='underinfluence_dow')  
print("Created underinfluence_dow Excel")

# # Average age of drivers who were at fault that involves another motor-vehicle 

average_age_of_drivers.to_csv("output/average_age_of_drivers.csv")  
print("Created average_age_of_drivers CSV")

Created num_of_collisions Excel
Created area_of_collision Excel
Created covid_collisions CSV
Created percent_collisions Excel
Created california_female_percentage Excel
Created collisions_per_year Excel
Created collisions_per_month Excel
Created underinfluence_dow Excel
Created average_age_of_drivers CSV


In [17]:
elapsed = timeit.default_timer() - start_time
print(elapsed)
print("Queries Finished")

1406.373389828
Queries Finished
